## Named Entity Recognition and Linking

**Introduction**

Named entity recognition (NER) is a form of natural language processing (NLP) that involves extracting and identifying essential information from text. Named Entity Linking (NEL) is the task to link entity mentions in text with their corresponding entities in a knowledge base.

**Aim**

To extract named entity like person, location, organizations, etc from a text and link it to their corresponding pages on knowledge base like Wikipedia, Wikidata, and DBpedia.

## **Fine-tuning BERT for named-entity recognition**

In this notebook, we are going to use **BERT** and **BART** which is included in the HuggingFace's transformer library. This model has BERT as its base architecture, with a token classification head on top, allowing it to make predictions at the token level, rather than the sequence level. Named entity recognition is typically treated as a token classification problem, so that's what we are going to use it for.

#### **Importing Python Libraries and preparing the environment**

This notebook assumes that you have the following libraries installed:
* pandas
* numpy
* sklearn
* pytorch
* transformers
* seqeval

As we are running this in Google Colab, the only libraries we need to additionally install are transformers and seqeval (GPU version):

In [ ]:
!pip install transformers seqeval[gpu]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import helper_functions as hf
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM
from transformers import pipeline

As deep learning can be accellerated a lot using a GPU instead of a CPU, make sure you can run this notebook in a GPU runtime (which Google Colab provides for free! - check "Runtime" - "Change runtime type" - and set the hardware accelerator to "GPU").

We can set the default device to GPU using the following code (if it prints "cuda", it means the GPU has been recognized):

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

#### **Downloading and preprocessing the data**
Named entity recognition (NER) uses a specific annotation scheme at the *word* level. An annotation scheme that is widely used is called IOB-tagging, which stands for Inside-Outside-Beginning. Each tag indicates whether the corresponding word is *inside*, *outside* or at the *beginning* of a specific named entity. The reason this is used is because named entities usually comprise more than 1 word.

Let's have a look at an example. If you have a sentence like "Barack Obama was born in Hawaï", then the corresponding tags would be   [B-PERS, I-PERS, O, O, O, B-GEO]. B-PERS means that the word "Barack" is the beginning of a person, I-PERS means that the word "Obama" is inside a person, "O" means that the word "was" is outside a named entity, and so on. So one typically has as many tags as there are words in a sentence.



In [ ]:
data = pd.read_csv("ner_datasetreference.csv", encoding='unicode_escape')
data.head()

Let's check how many sentences and words (and corresponding tags) there are in this dataset:

In [ ]:
data.count()

As we can see, there are approximately 48,000 sentences in the dataset, comprising more than 1 million words and tags (quite huge!). This corresponds to approximately 20 words per sentence.

Let's have a look at the different NER tags, and their frequency:

In [ ]:
entities_to_remove = ["B-art", "I-art", "B-eve", "I-eve", "B-nat", "I-nat"]
data = data[~data.Tag.isin(entities_to_remove)]
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies = data.Tag.value_counts()
frequencies

We create 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook.

In [ ]:
labels_to_ids = {k: v for v, k in enumerate(data.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(data.Tag.unique())}
labels_to_ids

Let's group the words and corresponding tags by sentence:

In [ ]:
data = data.fillna(method='ffill')
data.head()

In [ ]:
data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.head()

Let's only keep the "sentence" and "word_labels" columns, and drop duplicates:

In [ ]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

In [ ]:
len(data)

Let's verify that a random sentence and its corresponding tags are correct:

In [ ]:
data.iloc[41].sentence

In [ ]:
data.iloc[41].word_labels

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

We will now define the dataset class in which we will take the text alongwith the tokenizer and the maximum length.

        class dataset(Dataset):
          def __init__(self, dataframe, tokenizer, max_len):
            self.len = len(dataframe)
            self.data = dataframe
            self.tokenizer = tokenizer
            self.max_len = max_len

          def __getitem__(self, index):
            sentence = self.data.sentence[index].strip().split()
            word_labels = self.data.word_labels[index].split(",")

          def __init__(self, dataframe, tokenizer, max_len):
            self.len = len(dataframe)
            self.data = dataframe
            self.tokenizer = tokenizer
            self.max_len = max_len

          def __getitem__(self, index):
            sentence = self.data.sentence[index].strip().split()
            word_labels = self.data.word_labels[index].split(",")
        
            encoding = self.tokenizer(sentence,
                             is_pretokenized=True,
                             return_offsets_mapping=True,
                             padding='max_length',
                             truncation=True,
                             max_length=self.max_len)

            labels = [labels_to_ids[label] for label in word_labels]
            encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

            i = 0
            for idx, mapping in enumerate(encoding["offset_mapping"]):
              if mapping[0] == 0 and mapping[1] != 0:
                encoded_labels[idx] = labels[i]
                i += 1

            item = {key: torch.as_tensor(val) for key, val in encoding.items()}
            item['labels'] = torch.as_tensor(encoded_labels)

          return item

        def __len__(self):
          return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split of the dataset:

In [ ]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = hf.dataset(train_dataset, tokenizer, MAX_LEN, labels_to_ids, ids_to_labels)
testing_set = hf.dataset(test_dataset, tokenizer, MAX_LEN, labels_to_ids, ids_to_labels)

#### **Training set token view**

In [ ]:
training_set[0]

Let's verify that the input ids and corresponding targets are correct:

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

Now, let's define the corresponding PyTorch dataloaders:

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Then, we move the model to the GPU.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
modelNER = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
tokenizerNEL = AutoTokenizer.from_pretrained("facebook/mgenre-wiki")
modelNEL = AutoModelForSeq2SeqLM.from_pretrained("facebook/mgenre-wiki").eval()
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)


#### **Training the model**

The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83.

Let's verify this:



In [ ]:
inputs = training_set[2]
print(inputs)
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

Next, we define the optimizer. Here, we are just going to use a default learning rate.

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function.

    def train(epoch):
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []
        model.train()

        for idx, batch in enumerate(training_loader):

          ids = batch['input_ids'].to(device, dtype = torch.long)
          mask = batch['attention_mask'].to(device, dtype = torch.long)
          labels = batch['labels'].to(device, dtype = torch.long)

          loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
          tr_loss += loss.item()

          nb_tr_steps += 1
          nb_tr_examples += labels.size(0)

          if idx % 100==0:
              loss_step = tr_loss/nb_tr_steps
              print(f"Training loss per 100 training steps: {loss_step}")

          flattened_targets = labels.view(-1)
          active_logits = tr_logits.view(-1, model.num_labels)
          flattened_predictions = torch.argmax(active_logits, axis=1)

          active_accuracy = labels.view(-1) != -100

          labels = torch.masked_select(flattened_targets, active_accuracy)
          predictions = torch.masked_select(flattened_predictions, active_accuracy)

          tr_labels.extend(labels)
          tr_preds.extend(predictions)

          tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
          tr_accuracy += tmp_tr_accuracy

          torch.nn.utils.clip_grad_norm_(
              parameters=model.parameters(), max_norm=MAX_GRAD_NORM
          )

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      epoch_loss = tr_loss / nb_tr_steps
      tr_accuracy = tr_accuracy / nb_tr_steps
      print(f"Training loss epoch: {epoch_loss}")
      print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    hf.train(epoch, model)

#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits.

    def valid(model, testing_loader):
      # put model in evaluation mode
      model.eval()

      eval_loss, eval_accuracy = 0, 0
      nb_eval_examples, nb_eval_steps = 0, 0
      eval_preds, eval_labels = [], []

      with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

      labels = [ids_to_labels[id.item()] for id in eval_labels]
      predictions = [ids_to_labels[id.item()] for id in eval_preds]

      eval_loss = eval_loss / nb_eval_steps
      eval_accuracy = eval_accuracy / nb_eval_steps
      print(f"Validation Loss: {eval_loss}")
      print(f"Validation Accuracy: {eval_accuracy}")

      return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [ ]:
labels, predictions = hf.valid(model, testing_loader, ids_to_labels)

However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library:

In [ ]:
from seqeval.metrics import classification_report

print(classification_report(labels, predictions))

Performance already seems quite good, but note that we've only trained for 1 epoch. An optimal approach would be to perform evaluation on a validation set while training to improve generalization.

In [ ]:
inputs = tokenizerNEL(sentence.split(),
                    is_pretokenized=True,
                    return_offsets_mapping=True,
                    padding='max_length',
                    truncation=True,
                    max_length=MAX_LEN,
                    return_tensors="pt")

ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
outputs = model(ids, attention_mask=mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

prediction = []
for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
  #only predictions on first word pieces are important
  if mapping[0] == 0 and mapping[1] != 0:
    prediction.append(token_pred[1])
  else:
    continue

print(sentence.split())
print(prediction)

#### **Saving the model for future use**

Finally, let's save the vocabulary (.txt) file, model weights (.bin) and the model's configuration (.json) to a directory, so that both the tokenizer and model can be re-loaded using the `from_pretrained()` class method.


In [ ]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')
print('This tutorial is completed')